In [1]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime

In [2]:
envio_bq = pd.read_csv('resultados_fipe.csv', sep=',')
envio_bq['Valor'] = envio_bq['Valor'].apply(lambda x: float(x.replace('R$', '').replace('.', '').replace(',', '.')))
envio_bq['data_hora_atualizacao'] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

num_rows = envio_bq.shape[0]
num_rows

236924

In [3]:
key_path = "a.json"
credentials = service_account.Credentials.from_service_account_file(key_path)

In [4]:
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [5]:
envio_bq.dtypes

Marca                     object
Modelo                    object
Ano                        int64
Valor                    float64
Combustível               object
Código Fipe               object
Mês Referência            object
Autenticação              object
Tipo Veículo               int64
Sigla Combustível         object
Data Consulta             object
data_hora_atualizacao     object
dtype: object

In [6]:
# change the collumns names using a dic reference
new_collumns_name = {
    'Marca': 'marca',
    'Modelo': 'modelo',
    'Ano': 'ano',
    'Valor': 'valor',
    'Combustível': 'combustivel',
    'Código Fipe': 'codigo_fipe',
    'Mês Referência': 'mes_referencia',
    'Autenticação': 'autenticacao',
    'Tipo Veículo': 'tipo_veiculo',
    'Sigla Combustível': 'sigla_combustivel',
    'Data Consulta': 'data_hora_consulta',
    'data_hora_atualizacao': 'data_hora_atualizacao'
}

envio_bq = envio_bq.rename(columns=new_collumns_name)

In [7]:
project_id = 'leiloes-391501'
dataset_name = 'daniboy'
table_name = 'raw-precos-fipe-historico'

schema = [
    bigquery.SchemaField("marca", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("modelo", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("ano", "INT64", mode="NULLABLE"),
    bigquery.SchemaField("valor", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("combustivel", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("codigo_fipe", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("mes_referencia", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("autenticacao", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("tipo_veiculo", "INT64", mode="NULLABLE"),
    bigquery.SchemaField("sigla_combustivel", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("data_hora_consulta", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("data_hora_atualizacao", "STRING", mode="NULLABLE")
]

In [8]:
table_ref = client.dataset(dataset_name).table(table_name)
try:
    client.get_table(table_ref)
    print('A tabela já existe. Os novos dados serão adicionados a ela.')
except:
    print('A tabela não existe. Uma nova tabela será criada.')
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print('Tabela criada com sucesso.')

A tabela não existe. Uma nova tabela será criada.
Tabela criada com sucesso.


In [9]:
job_config = bigquery.LoadJobConfig(schema=schema)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

load_job = client.load_table_from_dataframe(envio_bq, table_ref, job_config=job_config)
print("Starting job {}".format(load_job.job_id))

load_job.result()
print("Job finished.")

destination_table = client.get_table(table_ref)
print("Loaded {} rows.".format(destination_table.num_rows))

Starting job 9f96efa4-b995-4800-9cd2-b527f389ad98
Job finished.
Loaded 236924 rows.


In [10]:
query = 'SELECT * FROM `leiloes-391501.daniboy.raw-precos-fipe-historico`'

dados_conferencia = client.query(query)
dados_conferencia = dados_conferencia.result().to_dataframe()

In [11]:
dados_conferencia

,marca,modelo,ano,valor,combustivel,codigo_fipe,mes_referencia,autenticacao,tipo_veiculo,sigla_combustivel,data_hora_consulta,data_hora_atualizacao
0,Acura,Integra GS 1.8,1992,11808.0,Gasolina,038003-2,julho de 2023,gsr806m66z,1,G,"sábado, 22 de julho de 2023 20:11",23/07/2023 13:21:27
1,Acura,Integra GS 1.8,1991,10766.0,Gasolina,038003-2,julho de 2023,gdkj7jbfsp,1,G,"sábado, 22 de julho de 2023 20:11",23/07/2023 13:21:27
2,Acura,Legend 3.2/3.5,1998,27006.0,Gasolina,038002-4,julho de 2023,njy9ddvlkp,1,G,"sábado, 22 de julho de 2023 20:11",23/07/2023 13:21:27
3,Acura,Legend 3.2/3.5,1997,23415.0,Gasolina,038002-4,julho de 2023,l5v2lg5818,1,G,"sábado, 22 de julho de 2023 20:11",23/07/2023 13:21:27
4,Acura,Legend 3.2/3.5,1996,22581.0,Gasolina,038002-4,julho de 2023,lv2yf0pqdj,1,G,"sábado, 22 de julho de 2023 20:11",23/07/2023 13:21:27
...,...,...,...,...,...,...,...,...,...,...,...,...
236919,GM - Chevrolet,Vectra GT 2.0 MPFI 8V FlexPower Mec.,2011,36150.0,Gasolina,004343-5,setembro de 2022,rz5vt9spn5fx,1,G,"domingo, 23 de julho de 2023 12:44",23/07/2023 13:21:27
236920,GM - Chevrolet,Vectra GT 2.0 MPFI 8V FlexPower Mec.,2010,33192.0,Gasolina,004343-5,setembro de 2022,qwhqdppnx2p,1,G,"domingo, 23 de julho de 2023 12:44",23/07/2023 13:21:27
236921,GM - Chevrolet,Vectra GT 2.0 MPFI 8V FlexPower Mec.,2009,32223.0,Gasolina,004343-5,setembro de 2022,qh4h4hfqtr8q,1,G,"domingo, 23 de julho de 2023 12:44",23/07/2023 13:21:27
236922,GM - Chevrolet,Vectra GT 2.0 MPFI 8V FlexPower Mec.,2008,29263.0,Gasolina,004343-5,setembro de 2022,pdfm4820kb5h,1,G,"domingo, 23 de julho de 2023 12:44",23/07/2023 13:21:27
